### Synopsis
This code averages and reduces g2 for temperature ramping (up and down) of protein H02

In [1]:

import numpy as np
import matplotlib.pyplot as plt
import h5py
import pickle

import sys
sys.path.append('../common')
from utils import process_group

fpath1 = '/home/8ididata/2021-2/babnigg202107_2/cluster_results_QZ'
fpath2 ='/home/8ididata/2022-1/babnigg202203/cluster_results_QZ'
coef_sam = 6.93e4  # Absolute scattering cross-section coefficient for the samples
coef_buf = 7.62e4  # Absolute scattering cross-section coefficient for the buffer


### Averaging of SA-XPCS results, temperature ramp up

In [ ]:


avg_ramp_up, t_el, ql_dyn, ql_sta = process_group(group='B039',
                                              num_sections=10,
                                              prefix=fpath1, 
                                              zone_idx='auto',
                                              num_cores=24)


### Averaging of SA-XPCS results, temperature ramp down

In [ ]:

avg_ramp_down, _, _, _ = process_group(group='B040',
                                 num_sections=10, 
                                 prefix=fpath1,
                                 zone_idx='auto',
                                 num_cores=24)


### Averaging of background 

In [ ]:

avg_bg, _, _, _ = process_group(group='D029',
                                 num_sections=1, 
                                 prefix='/home/8ididata/2021-2/babnigg202107_2/cluster_results_QZ',
                                 zone_idx='auto',
                                 num_cores=24)


### Data reduction

Subtracting I(Q) with background, converting to absolute scattering cross-section, and dump to pickle file for plotting

In [ ]:

for ii in range(len(avg_ramp_up)):
    avg_ramp_up[ii]["saxs_1d"] = coef_sam*avg_ramp_up[ii]["saxs_1d"] - coef_buf*avg_bg[0]["saxs_1d"]

for ii in range(len(avg_ramp_down)):
    avg_ramp_down[ii]["saxs_1d"] = coef_sam*avg_ramp_down[ii]["saxs_1d"] - coef_buf*avg_bg[0]["saxs_1d"]

with open('checkpoint.pickle', 'wb') as f:  # 'wb' for binary write mode
    pickle.dump(avg_ramp_up, f)  # Dump the object to the file
    pickle.dump(avg_ramp_down, f)
    pickle.dump(t_el, f)
    pickle.dump(ql_sta, f)
    pickle.dump(ql_dyn, f)
